In [9]:
import pandas as pd
import numpy as np
from PIL import Image
import os
from keras import *
from keras.models import Sequential,Model
from keras.layers import BatchNormalization,Embedding,Dense,Dropout,Conv2D,LSTM,MaxPooling2D,Flatten,MaxPooling1D,TimeDistributed,Reshape
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
import os 
import tifffile as tiff
from sklearn.metrics import f1_score
from keras.utils import np_utils,Sequence
np.random.seed(7)

os.chdir("../../")

In [4]:
class ImageDataGenerator(Sequence):
    
    def __init__(self, x_metadata, y_metadata, batch_size, crop_size):
        self.x = x_metadata
        self.y = y_metadata
        self.batch_size = batch_size
        self.cp = crop_size
    
    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))
    
    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]
        
        arr = []
        for file_name in batch_x:
            arr.append(np.array(np.array(tif.imread(file_name), dtype=int)/255.0)[self.cp:-self.cp,self.cp:-self.cp,:])
            
        return np.array(arr), np.array(batch_y) 

In [2]:
def crop_center(img,cropx,cropy):
    nparray = np.zeros((img.shape[0],img.shape[1]/2,img.shape[2]/2),dtype=np.float32)
    for i in range(img.shape[0]):
        y,x = img.shape[1],img.shape[2]
        startx = x//2-(cropx//2)
        starty = y//2-(cropy//2)    
        nparray[i] = img[i][starty:starty+cropy,startx:startx+cropx]
    return nparray

In [8]:
nparray = np.zeros(shape=(256*10,33,32,32),dtype=np.float32)
indices = []
counter=0
for j in os.listdir("Data/patchTrain/")[:10]:
    for i in os.listdir("Data/patchTrain/"+str(j)+"/"):
        nparray[counter] = crop_center(tiff.imread("Data/patchTrain/"+j+"/"+i),32,32)/255.0
        indices += [int(i.split("_")[1].split(".")[0])]
        counter+=1

df = pd.read_csv("occurrences_train.csv",sep=';', error_bad_lines=False)
df = df[["class","order","family","genus","species"]]
df = df.iloc[indices,:]
df.reset_index(drop=True,inplace=True)

encoder = LabelEncoder()
encoder = encoder.fit(df['class'])
df ['class'] = encoder.transform(df['class'])

encoder = LabelEncoder()
encoder = encoder.fit(df['order'])
df ['order'] = encoder.transform(df['order'])

encoder = LabelEncoder()
encoder = encoder.fit(df['family'])
df ['family'] = encoder.transform(df['family'])

encoder = LabelEncoder()
encoder = encoder.fit(df['genus'])
df ['genus'] = encoder.transform(df['genus'])

encoder = LabelEncoder()
encoder = encoder.fit(df['species'])
df ['species'] = encoder.transform(df['species'])

FileNotFoundError: [Errno 2] No such file or directory: 'Data/patchTrain/'

In [178]:
input_arr = np.zeros((df.shape[0],1))
for i in range(df.shape[0]):
    for j in range(df.iloc[i,:-4].shape[0]):x
        input_arr[i][j] = df.iloc[i,j]

In [180]:
input_arr

array([[3.],
       [3.],
       [3.],
       ...,
       [3.],
       [3.],
       [3.]])

In [273]:
from keras import backend as K

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        '''
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        '''
        return tf.metrics.recall(y_true,y_pred)

    def precision(y_true, y_pred):
        '''
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        '''
        return tf.metrics.precision(y_true,y_pred)
    
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return tf.multiply ( tf.constant(2), tf.divide (tf.add(precision,recall),tf.add(precision,recall,tf.backend.keras.epsilon) ) )
    #return 2*((precision*recall)/(precision+recall+tf.backend.keras.epsilon))

    
model = Sequential()
model.add(Embedding(500,32,input_length=1))
model.add(LSTM(1,input_shape=input_arr.shape[1:],return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(4,return_sequences=True))
model.summary()
model.compile(loss='mae',optimizer='Adam',metrics=['accuracy',f1])

Exception KeyboardInterrupt in <function _remove at 0x7fbdb3ace8c0> ignored


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_30 (Embedding)     (None, 1, 32)             16000     
_________________________________________________________________
lstm_79 (LSTM)               (None, 1, 1)              136       
_________________________________________________________________
dropout_81 (Dropout)         (None, 1, 1)              0         
_________________________________________________________________
lstm_80 (LSTM)               (None, 1, 4)              96        
Total params: 16,232
Trainable params: 16,232
Non-trainable params: 0
_________________________________________________________________


AttributeError: 'module' object has no attribute 'backend'

In [188]:
input_arr

array([[3.],
       [3.],
       [3.],
       ...,
       [3.],
       [3.],
       [3.]])

In [189]:
model.fit(input_arr,df.values[:,1:],batch_size=32,epochs=5000)

ValueError: Error when checking target: expected lstm_59 to have 3 dimensions, but got array with shape (2560, 4)

In [151]:
input_arr[:,:,0][:,:,np.newaxis]

array([[[3.]],

       [[3.]],

       [[3.]],

       ...,

       [[3.]],

       [[3.]],

       [[3.]]])

In [5]:
from keras import backend as K

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        '''
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        '''
        return tf.metrics.recall(y_true,y_pred)

    def precision(y_true, y_pred):
        '''
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        '''
        return tf.metrics.precision(y_true,y_pred)
    
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return tf.multiply ( tf.constant(2.0), tf.divide (tf.add(precision,recall),tf.add(precision,recall) ) )
    #return 2*((precision*recall)/(precision+recall+tf.backend.keras.epsilon))

model = Sequential()
model.add(Conv2D(filters=32,kernel_size=(2,2),activation='relu',input_shape=(33,32,32)))
model.add(Conv2D(filters=64,kernel_size=(4,4),activation='relu'))
model.add(Conv2D(filters=128,kernel_size=(6,6),activation='relu'))
model.add(Conv2D(filters=256,kernel_size=(10,10),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Reshape((7*7,256)))
model.add(Dropout(0.2))
model.add(LSTM(4))
model.add(Dropout(0.2))
model.add(Dense(4))
model.compile(loss='mae',optimizer='Adam',metrics=['accuracy','categorical_accuracy'])

In [152]:
model.predict(input_arr[:,:,0][:,:,np.newaxis])

ValueError: Error when checking : expected lstm_40_input to have shape (None, 1, 4) but got array with shape (2560, 1, 1)

In [6]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 31, 32)        4128      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 29, 28, 64)        32832     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 24, 23, 128)       295040    
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 15, 14, 256)       3277056   
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 256)         0         
_________________________________________________________________
reshape_1 (Reshape)          (None, 49, 256)           0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 49, 256)           0         
__________

In [18]:
y_coarse = pd.get_dummies(df.iloc[:,0])
y_fine = pd.get_dummies(df.iloc[:,1])

In [32]:
img_input = Input(shape=(33,32,32), name='input')

In [ ]:
#--- block 1 ---
x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1')(img_input)
x = BatchNormalization()(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

#--- block 2 ---
x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
x = BatchNormalization()(x)
x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

#--- coarse 1 branch ---
c_1_bch = Flatten(name='c1_flatten')(x)
c_1_bch = Dense(256, activation='relu', name='c1_fc_cifar10_1')(c_1_bch)
c_1_bch = BatchNormalization()(c_1_bch)
c_1_bch = Dropout(0.5)(c_1_bch)
c_1_bch = Dense(256, activation='relu', name='c1_fc2')(c_1_bch)
c_1_bch = BatchNormalization()(c_1_bch)
c_1_bch = Dropout(0.5)(c_1_bch)
c_1_pred = Dense(5, activation='softmax', name='c1_predictions_cifar10')(c_1_bch)

#--- block 4 ---
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
x = BatchNormalization()(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

#--- fine block ---
x = Flatten(name='flatten')(x)
x = Dense(1024, activation='relu', name='fc_cifar10_1')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation='relu', name='fc2')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
fine_pred = Dense(27, activation='softmax', name='predictions_cifar10')(x)

model = Model(input=img_input, output=[c_1_pred, fine_pred], name='medium_dynamic')

#----------------------- compile and fit ---------------------------
sgd = optimizers.SGD(lr=0.003, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', 
              optimizer='Adam', 
              metrics=['accuracy'])

model.fit(nparray, [ y_coarse.values , y_fine.values ],
          batch_size=32,
          epochs=100,
          verbose=1)


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:42: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=[<tf.Tenso..., name="medium_dynamic", inputs=Tensor("in...)`


Epoch 1/100
2560/2560 [==============================] - 287s - loss: 5.7731 - c1_predictions_cifar10_loss: 1.7878 - predictions_cifar10_loss: 3.9854 - c1_predictions_cifar10_acc: 0.4238 - predictions_cifar10_acc: 0.1367   
Epoch 2/100
2560/2560 [==============================] - 238s - loss: 4.6347 - c1_predictions_cifar10_loss: 1.1482 - predictions_cifar10_loss: 3.4865 - c1_predictions_cifar10_acc: 0.6430 - predictions_cifar10_acc: 0.1625   
Epoch 3/100
2560/2560 [==============================] - 232s - loss: 4.1609 - c1_predictions_cifar10_loss: 0.9090 - predictions_cifar10_loss: 3.2519 - c1_predictions_cifar10_acc: 0.7406 - predictions_cifar10_acc: 0.1809   
Epoch 4/100
2560/2560 [==============================] - 234s - loss: 3.9104 - c1_predictions_cifar10_loss: 0.8299 - predictions_cifar10_loss: 3.0804 - c1_predictions_cifar10_acc: 0.7723 - predictions_cifar10_acc: 0.1789   
Epoch 5/100
2560/2560 [==============================] - 233s - loss: 3.8121 - c1_predictions_cifar10_lo

In [50]:
y_coarse.values

array([[0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0],
       ...,
       [0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0]], dtype=uint8)

In [7]:
df.values[:,1:]

NameError: name 'df' is not defined

In [ ]:
model.fit(nparray,df.values[:,1:],epochs=1000)

Epoch 1/1000
2560/2560 [==============================] - 487s - loss: 58.5739 - acc: 0.6059 - categorical_accuracy: 0.6059   
Epoch 2/1000
2560/2560 [==============================] - 436s - loss: 58.2572 - acc: 0.6355 - categorical_accuracy: 0.6355   
Epoch 3/1000
2560/2560 [==============================] - 437s - loss: 57.9708 - acc: 0.6152 - categorical_accuracy: 0.6152   
Epoch 4/1000
2560/2560 [==============================] - 447s - loss: 57.6738 - acc: 0.6125 - categorical_accuracy: 0.6125   
Epoch 5/1000
2112/2560 [=======================>......] - ETA: 91s - loss: 57.4983 - acc: 0.6264 - categorical_accuracy: 0.6264

In [254]:
model.fit(nparray,df.values[:,1:],epochs=1000)

Epoch 1/1000
2560/2560 [==============================] - 546s - loss: 57.8818 - acc: 0.2672   
Epoch 2/1000
2560/2560 [==============================] - 538s - loss: 57.6013 - acc: 0.6074   
Epoch 3/1000
2560/2560 [==============================] - 455s - loss: 57.3081 - acc: 0.6191   
Epoch 4/1000
2560/2560 [==============================] - 468s - loss: 57.0314 - acc: 0.6145   
Epoch 5/1000
2560/2560 [==============================] - 469s - loss: 56.7828 - acc: 0.6063   
Epoch 6/1000
2560/2560 [==============================] - 451s - loss: 56.5042 - acc: 0.2422   
Epoch 7/1000
2560/2560 [==============================] - 439s - loss: 56.2777 - acc: 0.3426   
Epoch 8/1000
2560/2560 [==============================] - 452s - loss: 56.0032 - acc: 0.6113   
Epoch 9/1000
2560/2560 [==============================] - 436s - loss: 55.7958 - acc: 0.5941   
Epoch 10/1000
2560/2560 [==============================] - 437s - loss: 55.5250 - acc: 0.6258   
Epoch 11/1000
2560/2560 [==============

KeyboardInterrupt: 

In [255]:
ypred = model.predict(nparray)

In [256]:
ypred

array([[5.258827 , 6.6775947, 6.8487415, 6.8486757],
       [5.258827 , 6.6775947, 6.8487415, 6.8486757],
       [5.258827 , 6.6775947, 6.8487415, 6.8486757],
       ...,
       [5.258827 , 6.6775947, 6.8487415, 6.8486757],
       [5.258827 , 6.6775947, 6.8487415, 6.8486757],
       [5.258827 , 6.6775947, 6.8487415, 6.8486757]], dtype=float32)

In [260]:
np.max(ypred[:,:]) , np.min(ypred[:,0])

(6.8487415, 5.258827)

In [5]:
for i in os.listdir("Data/patchTrain/")[:1000]:
    print i

86784
12032
31744
6144
99584
82944
105728
189952
177408
76288
42240
29440
208640
66048
50944
177664
206080
71168
40192
38400
214016
5632
85248
67840
123904
210432
83712
182784
142336
147456
39680
113920
111872
128512
98560
114176
216320
179712
124672
45056
52224
108032
44288
1536
185088
143616
200960
163072
186624
26112
80896
82688
139264
43776
101376
23040
170240
86272
27904
43008
96000
213760
19968
164864
28928
187392
14080
46848
208896
166144
24320
119040
113664
125952
60928
52480
14592
70656
214272
156416
29696
48128
96768
60416
97024
9728
42496
146944
72704
196608
66816
18944
68608
57600
90624
127744
214784
2560
160256
172288
35072
203264
94976
130048
16640
198144
103936
107776
66304
63744
55808
131072
186880
174592
101888
216064
155392
188928
82176
35328
80384
84992
140032
202240
123648
192256
80128
137216
128256
117760
27648
137984
45312
40960
61952
142080
139520
171264
87552
211712
59904
171776
129280
24064
206592
139776
196352
38656
123392
26368
44544
73984
186112
42752
1792
8

In [3]:
nparray = np.zeros(shape=(256*10,33,32,32),dtype=np.float32)
indices = []
counter=0
for j in os.listdir("Data/patchTrain/")[:10]:
    for i in os.listdir("Data/patchTrain/"+str(j)+"/"):
        nparray[counter] = crop_center(tiff.imread("Data/patchTrain/"+j+"/"+i),32,32)/255.0
        indices += [int(i.split("_")[1].split(".")[0])]
        counter+=1

df = pd.read_csv("Data/occurrences_train.csv",sep=';', error_bad_lines=False)
df = df[["class","order"]]
df = df.iloc[indices,:]
df.reset_index(drop=True,inplace=True)
encoder = LabelEncoder()
encoder = encoder.fit(df['class'])
df ['class'] = encoder.transform(df['class'])

encoder = LabelEncoder()
encoder = encoder.fit(df['order'])
df ['order'] = encoder.transform(df['order'])

In [4]:
df = pd.read_csv("Data/occurrences_train.csv",sep=';', error_bad_lines=False)
df = df[["class","order"]]
df = df.iloc[indices,:]
df.reset_index(drop=True,inplace=True)
encoder = LabelEncoder()
encoder = encoder.fit(df['class'])
df ['class'] = encoder.transform(df['class'])

encoder = LabelEncoder()
encoder = encoder.fit(df['order'])
df ['order'] = encoder.transform(df['order'])

/usr/local/lib/python2.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (36,41) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
nparray

array([[[[0.5019608 , 0.5019608 , 0.5058824 , ..., 0.50980395,
          0.5058824 , 0.50980395],
         [0.5019608 , 0.5019608 , 0.5058824 , ..., 0.50980395,
          0.50980395, 0.5058824 ],
         [0.5019608 , 0.5019608 , 0.5019608 , ..., 0.5058824 ,
          0.5058824 , 0.50980395],
         ...,
         [0.        , 0.        , 0.5137255 , ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.5137255 , 0.50980395, ..., 0.        ,
          0.5176471 , 0.5176471 ],
         [0.        , 0.5137255 , 0.50980395, ..., 0.5137255 ,
          0.5137255 , 0.5137255 ]],

        [[0.7176471 , 0.7176471 , 0.7176471 , ..., 0.7176471 ,
          0.7176471 , 0.7176471 ],
         [0.7176471 , 0.7176471 , 0.7176471 , ..., 0.7176471 ,
          0.7176471 , 0.7176471 ],
         [0.7137255 , 0.7137255 , 0.7176471 , ..., 0.7176471 ,
          0.7176471 , 0.7176471 ],
         ...,
         [0.        , 0.        , 0.73333335, ..., 0.        ,
          0.        , 0. 

In [15]:
print nparray.shape , df.shape

(5120, 33, 32, 32) (5120, 2)


In [81]:
model.fit(nparray,df.values,epochs=1,batch_size=128)

ValueError: Error when checking target: expected dense_23 to have shape (None, 1) but got array with shape (5120, 2)

In [11]:
df.values

array([[ 3, 24],
       [ 3, 24],
       [ 3, 11],
       ...,
       [ 3,  3],
       [ 3,  3],
       [ 3,  3]])

In [29]:
ypred = model.predict(nparray)

In [39]:
np.max(df.values[:,0]) , np.min(df.values[:,0])

(5, 0)

In [37]:
np.max(ypred[:,0]) , np.min(ypred[:,0])

(0.8106967, 0.33322722)

In [33]:
ypred

array([[0.58821476, 3.3585274 ],
       [0.56894076, 3.3399484 ],
       [0.6069794 , 3.3722222 ],
       ...,
       [0.5786255 , 3.330465  ],
       [0.63559383, 3.3927653 ],
       [0.62512434, 3.3843596 ]], dtype=float32)